# SoS utilities
For simulation and data analysis of the CNV project.

## Workflows

In [1]:
[varbvs_1]
depends: Py_Module("pandasql"), Py_Module('fisher'), Py_Module('plotly')
parameter: sample_size = 2000
parameter: seed = [12]
parameter: prevalence = [0.01]
parameter: gamma = [(3,1), (5,1), (9,1)]
parameter: n_causal_gene = [100, 200]
parameter: name = '../analysis/data/del_sim'
input: for_each = ['gamma', 'prevalence', 'n_causal_gene', 'seed']
output: "${name}_%s.data.pkl" % "shape{}_scale{}_p{}_N{}_M{}_{}".format("${_gamma[0]}", "${_gamma[1]}", "${_prevalence}", int("${sample_size}") * 2, "${_n_causal_gene}", "${_seed}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from utils import Environment, run_simulation 
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['n_causal_gene'] = ${_n_causal_gene}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, "shape{}_scale{}_p{}_N{}_M{}_{}".\
               format(${_gamma[0]}, ${_gamma[1]}, ${_prevalence}, ${sample_size} * 2, ${_n_causal_gene}, ${_seed}))

[varbvs_2]
depends: Py_Module("feather-format")
input: group_by = 'single'
output: "${_input!n}.feather"
task:
python:
import sys, os
sys.path.append(os.getcwd())
import pickle, feather
import pandas as pd
import numpy as np
from utils import Environment, load_reference_gene 
args = Environment()
#
dat = pickle.load(open(${_input!r}, "rb"))
ref = load_reference_gene(args["refgene_file"])
# create regression data
genes = pd.Series(list(set(ref['gene_name'])))
regression_data = np.array([np.array(genes.isin(item["gene_name"]), dtype = float) for item in dat['case'] + dat['ctrl']])
regression_data = np.hstack((np.matrix([1] * len(dat['case']) + [0] * len(dat['ctrl'])).T, regression_data))
mask = ~np.ravel((regression_data==0).all(0))
# fix next line according to feather
regression_data = regression_data[:, mask]
# save to disk
feather.write_dataframe(pd.DataFrame(regression_data, columns = ['phenotype'] + genes[mask[1:]].tolist()), ${_output!r})

[varbvs_3]
depends: R_library("varbvs"), R_library("feather")
input: group_by = "single"
output: "${_input!nn}.varbvs.rds"
task:
R:
dat = as.matrix(feather::read_feather(${_input!r}))
# FIXME: need to check if the usage is correct
fit = varbvs::varbvs(dat[,-1], NULL, dat[,1], family = "binomial", logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
fit$summary = summary(fit)
fit$w = varbvs::normalizelogweights(fit$logw)
fit$pip = fit$alpha %*% c(fit$w)
fit$beta = fit$mu %*% c(fit$w)
fit$var_names = colnames(dat[-1])
saveRDS(fit, ${_output!r})

[dap_1]
depends: Py_Module("pandasql")
parameter: sample_size = 10
parameter: seed = [999]
parameter: prevalence = [0.01]
parameter: shape = [25]
parameter: scale = [1]
parameter: prefix = "data/tmp.test"
parameter: ncpu = 4
input: for_each = ["shape", "scale", "prevalence", "seed"]
output: "${prefix!a}_%s.data.pkl" % "{}_shape_{}".format(int("${sample_size}")*2, "${_shape}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from utils import Environment, run_simulation
args = Environment()
args['odds_ratio_params']['shape'] = ${_shape}
args['odds_ratio_params']['scale'] = ${_scale}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['output'] = ${_output!nnr}
args['ncpu'] = ${ncpu}
args.seed = ${_seed}
run_simulation(args)

[dap_2]
depends: Py_Module("feather-format")
input: group_by = 'single'
output: "${_input!n}.blocks.pkl"
task:
python:
import sys, os, pickle
import numpy as np
sys.path.append(os.getcwd())
from utils import pkl_to_matrix, save_data
dat = pkl_to_matrix(${_input!r}, make_block = True, dtype = np.uint8)
save_data(dat, ${_output!r})

## Simulation for variable selection methods
This pipeline implements simulation and data analysis using 
* Carbonetto & Stephens 2012 (varbvs)
* Wen et al 2016 (DAP)

### DAP data simulation

**For calcium pathway simulation**

In [2]:
%sosrun dap -J 1 -j 1 --sample_size 20 --seed 12 --shape 25 --prefix "../analysis/data/calcium_pathway" -v 3

DEBUG: Workflow dap created with 2 sections: dap_1, dap_2
DEBUG: Args "${prefix!a}_%s.data.pkl" % "N{}_p{}".format(int("${sample_size}")*2, "${_prevalence}")
 cannot be determined: name 'prefix' is not defined
DEBUG: Args "${_input!n}.blocks.pkl"
 cannot be determined: name '_input' is not defined
DEBUG: Args "${name}_%s.data.pkl" % "shape{}_scale{}_p{}_N{}_M{}_{}".format("${_gamma[0]}", "${_gamma[1]}", "${_prevalence}", int("${sample_size}") * 2, "${_n_causal_gene}", "${_seed}")
 cannot be determined: name 'name' is not defined
DEBUG: Args "${_input!n}.feather"
 cannot be determined: name '_input' is not defined
DEBUG: Args "${_input!nn}.varbvs.rds"
 cannot be determined: name '_input' is not defined
DEBUG: Args "${prefix!a}_%s.data.pkl" % "N{}_p{}".format(int("${sample_size}")*2, "${_prevalence}")
 cannot be determined: name 'prefix' is not defined
DEBUG: Args "${_input!n}.blocks.pkl"
 cannot be determined: name '_input' is not defined
DEBUG: Executing dap_1: 
DEBUG: input:    []
DEB

### Resume from simulated data
Input are simulation results, output is blocked data: basically run `dap_2`

In [14]:
[make_block]
input: ["../analysis/data/calcium_pathway_1000_shape_25.data.pkl", 
        "../analysis/data/calcium_pathway_500_shape_5.data.pkl", 
        "../analysis/data/calcium_pathway_500_shape_25.data.pkl",
        "../analysis/data/calcium_pathway_4000_shape_25.data.pkl", 
        "../analysis/data/calcium_pathway_20_shape_5.data.pkl"]
sos_run('dap:2')

In [21]:
%sosrun make_block -J 5

5 tasks completed: 1bc1 , b13d , ..., 2cda